In [26]:
import numpy as np
import pandas as pd
import math
import itertools
import datetime

In [27]:
def wczytanieMacierzy(rozmiar):
    file = 'Dane/data{}.txt'.format(rozmiar)
    df = pd.read_csv(file, nrows=2, header=None)
    name = df[df.index==0].values[0][0]
    n = int(df[df.index==1].values[0][0])

    df = pd.read_csv(file, skiprows=2, sep='\s+', header=None)
    arr = df.values
    return arr

In [28]:
def obliczenieCzasu(arr, n):
    start = datetime.datetime.now()
    C = {}
    for k in range(1, n):
        C[(1 << k, k)] = (arr[0][k], 0) #tworzymy drogę i zapisujemy dystans każdego wierzchołka z ostatnim
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            bits = 0
            for bit in subset:
                bits |= 1 << bit    # ustawiamy bity na 1 w miejscu, na który wskazuje nr wierzchołka z bierzącego subset, 
                                    # operacją 'or' dodajemy kolejne jedynki

            for k in subset:
                prev = bits & ~(1 << k) # usuwamy k'ty wierzchołekz bitu, 
                                        # prev to kombinacja, która już była (iteracja zwiększa rozmiar)

                res = []
                for m in subset:
                    if m == k:  # w celu uniknięcia ścieżki do siebie samej
                        continue
                    res.append((C[(prev, m)][0] + arr[m][k], m)) # nowy dystans oraz ostatni wierzchołek
                C[(bits, k)] = min(res)

    bits = (1<<n) - 2
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + arr[k][0], k))
    opt, parent = min(res)

    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    path.append(0)
    path.insert(0,0)

    duration = datetime.datetime.now() - start
    return duration.total_seconds()

In [33]:
def obliczenia(rozmiary, liczba_powtorzen):
    srednie_czasy = []
    najlepsze_czasy = []
    najgorsze_czasy = []
    
    for nr, rozmiar in enumerate(rozmiary):
        ilosc_powtorzen = liczba_powtorzen[nr]
        najlepszy_czas = 999999999
        najgorszy_czas = 0
        suma_czasow = 0
        
        for powtorzenie in range(ilosc_powtorzen):
            arr = wczytanieMacierzy(rozmiar)
            czas = obliczenieCzasu(arr, rozmiar)
            if czas < najlepszy_czas:
                najlepszy_czas = czas
            
            if czas > najgorszy_czas:
                najgorszy_czas = czas
                
            suma_czasow += czas
        
        sredni_czas = suma_czasow / ilosc_powtorzen
        najlepsze_czasy.append(najlepszy_czas)
        najgorsze_czasy.append(najgorszy_czas)
        srednie_czasy.append(sredni_czas)
    
    return najlepsze_czasy, najgorsze_czasy, srednie_czasy

In [34]:
rozmiary = [10, 11, 12, 13, 14, 15, 16, 17, 18, 21]
liczba_powtorzen = [100, 100, 100, 100, 100, 100, 100, 100, 50, 10]
#rozmiary = [10, 11, 12, 13, 14, 15, 16, 17, 18]
#liczba_powtorzen = [1,1,1,1,1,1,1,1,1]
najlepsze_czasy, najgorsze_czasy, srednie_czasy = obliczenia(rozmiary, liczba_powtorzen)

In [35]:
for i in range(len(rozmiary)):
    print('Rozmiar: ', rozmiary[i], ', liczba powtorzen: ', liczba_powtorzen[i], ', najgorsze_czasy: ', najgorsze_czasy[i], ', najlepszy czas: ', najlepsze_czasy[i], ', sredni czas: ', srednie_czasy[i])

Rozmiar:  10 , liczba powtorzen:  100 , najgorsze_czasy:  0.017951 , najlepszy czas:  0.00698 , sredni czas:  0.0082479
Rozmiar:  11 , liczba powtorzen:  100 , najgorsze_czasy:  0.0239 , najlepszy czas:  0.016955 , sredni czas:  0.019617379999999997
Rozmiar:  12 , liczba powtorzen:  100 , najgorsze_czasy:  0.086789 , najlepszy czas:  0.042885 , sredni czas:  0.04922837999999999
Rozmiar:  13 , liczba powtorzen:  100 , najgorsze_czasy:  0.187629 , najlepszy czas:  0.103722 , sredni czas:  0.11609431000000002
Rozmiar:  14 , liczba powtorzen:  100 , najgorsze_czasy:  0.330116 , najlepszy czas:  0.249332 , sredni czas:  0.27822375000000005
Rozmiar:  15 , liczba powtorzen:  100 , najgorsze_czasy:  0.832806 , najlepszy czas:  0.588426 , sredni czas:  0.64560918
Rozmiar:  16 , liczba powtorzen:  100 , najgorsze_czasy:  1.83954 , najlepszy czas:  1.365388 , sredni czas:  1.4814431699999993
Rozmiar:  17 , liczba powtorzen:  100 , najgorsze_czasy:  4.452333 , najlepszy czas:  3.173515 , sredni cz

In [36]:
PRD = []
for i in range(len(rozmiary)):
    PRD.append(100*(srednie_czasy[i] - najlepsze_czasy[i])/srednie_czasy[i])

In [37]:
df_dynamic_programming = pd.DataFrame({
    'rozmiar' : rozmiary,
    'liczba powtorzen' : liczba_powtorzen,
    'najlepszy czas' : najlepsze_czasy,
    'najgorszy czas' : najgorsze_czasy,
    'sredni czas' : srednie_czasy,
    'PRD' : PRD
})

In [38]:
df_dynamic_programming

,rozmiar,liczba powtorzen,najlepszy czas,najgorszy czas,sredni czas,PRD
0,10,100,0.006980,0.017951,0.008248,15.372398
1,11,100,0.016955,0.023900,0.019617,13.571537
2,12,100,0.042885,0.086789,0.049228,12.885616
3,13,100,0.103722,0.187629,0.116094,10.657120
4,14,100,0.249332,0.330116,0.278224,10.384358
5,15,100,0.588426,0.832806,0.645609,8.857244
6,16,100,1.365388,1.839540,1.481443,7.833927
7,17,100,3.173515,4.452333,3.512160,9.642062
8,18,50,7.360358,10.577747,8.160626,9.806451
9,21,10,99.890408,106.504056,101.805529,1.881157


In [44]:
df_dynamic_programming.to_csv('dynamic_programming.csv', encoding='utf-8', index=False)